# Pré-processamento de dados
<p>Este notebook foca na preparação dos dados antes de aplicarmos os algoritmos de aprendizado de máquina. <br>
O pré-processamento inclui etapas como o tratamento de valores ausentes, a conversão de tipos de dados, entre outros. <br>
Essas etapas buscam preparar os dados para análise e facilitar uma melhor performance dos modelos de machine learning.</p>

---
## Importe das bibliotecas e o banco de dados via arquivo local
Importação das bibliotecas que serão utilizadas ao longo do desenvolvimento do Colab.<br>
Pandas (pd) para análises de dataframes, Numpy (np) para cálculos estatísticos com processamento mais rápido, Matplotlib (plt) e Seaborn (sns) para plotação de gráficos.<br>
Além disso, há a importação do banco de dados e a definição de um dataframe (df) para armazenar o banco como dataframe através do método `read_csv()`.<br>
Na última linha, existe a utilização do método `info()`, que identifica o tipo de dado de cada variável e se existe valores nulos.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv('BASE DE SINISTRO UNIPAR BRADESCO.csv', decimal=',')

df.info()

---
## Etapas do pré-processamento
### Tratamento de missing values

Na seguinte célula, mesmo sabendo que não existiam valores nulos, realizamos o método `dropna()`, para garantir que não trabalharemos com valores ausentes.


In [ ]:
df = df.dropna()
df

### Remoção de colunas
A partir daqui, a equipe de Exploração dos Dados sugeriu a eliminação de algumas colunas que não seriam eficazes no processo de análise e que poderiam atrapalhar no processo de treinamento do modelo.

In [ ]:
df.drop('Nome Grupo Empresa',
  axis='columns', inplace=True)

df.drop('Codigo Grupo Empresa',
  axis='columns', inplace=True)

df.drop('Valor Usuario Sinistro',
  axis='columns', inplace=True)

df.drop('Apolice Sinistro',
  axis='columns', inplace=True)

In [ ]:
df

### Correção da coluna 'Nome Empresa Sinistro'
A equipe relatou que a coluna "Nome Empresa Sinistro" possui três valores distintos, mas dois desses valores representam a mesma empresa, diferenciando-se apenas pela formatação.

In [ ]:
df['Nome Empresa Sinistro'].value_counts()

Visto que a diferença está na presença de "." no final de um dos valores, na seguinte célula, acontece o método `replace()` para substituir os valores e juntá-los em uma mesma contagem da coluna.


In [ ]:
df = df.replace({"UNIPAR INDUPA DO BRASIL S.A": "UNIPAR INDUPA DO BRASIL S.A."})

In [ ]:
df

In [ ]:
df['Nome Empresa Sinistro'].value_counts()

### Remoção de valores da coluna 'Elegibilidade Sinistro'
Considerando que a empresa tem foco em desenvolver planos de saúde para seus funcionários, descartaremos a análise referente aos dados de saúde que sejam de origem de "DEPENDENTES" e "AGREGADOS".

In [ ]:
df['Elegibilidade Sinistro'].value_counts()

In [ ]:
df_remove_d = df.loc[(df['Elegibilidade Sinistro'] == 'DEPENDENTE') ]

df = df.drop(df_remove_d.index)

df


In [ ]:
df['Elegibilidade Sinistro'].value_counts()

In [ ]:
df_remove_a = df.loc[(df['Elegibilidade Sinistro'] == 'AGREGADO') ]

df = df.drop(df_remove_a.index)

df

In [ ]:
df['Elegibilidade Sinistro'].value_counts()

### Análise de dados por nível de valor
Considerando que estamos buscando os outliers da tabela de 'Valor Pago Sinistro', iremos fazer uma categorização baseada em uma contagem do valor do sinistro. Para isso, precisa-se criar um array com os valores que deseja-se usar como parâmetro de divisão. Logo em seguida, cria-se outro array nomeando as categorias desejadas. Então, através do método `cut()`, cria-se uma nova coluna (categoria_sinistro) utilizando dois parâmetros: *bins* ('caixas' com os valores) e *labels* (rótulos estabelecidos).

Utilizamos a seguinte classificação:
- 0-50: nível 1
- 50-100: nível 2
- 100-200: nível 3
- 200-500: nível 4
- 500-10000: nível 5
- 10000-infinito: nível 6

Escolhemos essa distribuição de valor pois percebemos que a grande maioria dos valores estavam no começo da tabela, ou seja, estavam aproximadamente entre 0 e 50.


In [ ]:
faixas_valor = [0, 50, 100, 200, 500, 10000, float("inf")]

categorias_valor = ["Nível 1", "Nível 2", "Nível 3", "Nível 4", "Nível 5", "Nível 6"]

df["Categoria Sinistro"] = pd.cut(df["Valor Pago Sinistro"], bins=faixas_valor, labels=categorias_valor)

df['Categoria Sinistro'].value_counts()

Antes de remover os outliers, iremos visualizar o valor médio dos sinistros para que possamos comparar com o valor pós-limpeza.

In [ ]:
df['Valor Pago Sinistro'].mean()

Visto que temos o obejtivo que remover outliers, iremos analisar cada uma dessas categorias e desenvolver um gráfico boxplot, que indentificará justamente os valores que são discrepantes.

É importante sabermos que o Boxplot é uma ferramenta que permite a visualização da distribuição dos dados e dos valores discrepantes dentro do grupo específico que está sendo analisado, ou seja, ela mostra para nós quais são os valores que seriam discrepantes dentro dos grupos que criamos.

In [ ]:
df.boxplot(['Valor Pago Sinistro'], by = ['Categoria Sinistro'])

A partir disso, conluímos que uma limpeza por categoria, começando por "extremamente alto" será o método mais eficiente. Portanto, começaremos por localizar os valores que se encaixam nessa categoria através do método `loc()` e atribí-los a uma variável.

In [ ]:
df_nivel6 = df.loc[(df['Categoria Sinistro'] == 'Nível 6') ]


A partir disso, criaremos uma variável que armazena o exato valor que se encontra no quartil de 75%.

In [ ]:
quartil = df_nivel6['Valor Pago Sinistro'].quantile(0.75)

quartil

Antes de criar uma nova categoria, dentro das já existentes, que separa os valores que estão a acima do valor do quartil 75% (os outliers), é preciso converter a coluna 'Categoria Sinistro' para o tipo Categorical. Além disso, é preciso também adicionar a nova categoria ('Nível 6 Intermediário') às categorias existentes. Esse processo se dá pelo método `cat.add_categories()`.


In [ ]:
df_nivel6['Categoria Sinistro'] = df_nivel6['Categoria Sinistro'].astype('category')

df_nivel6['Categoria Sinistro'] = df_nivel6['Categoria Sinistro'].cat.add_categories(['Nível 6 Intermediário'])

Na seguinte célula, atribuiremos a condição de ser maior que o valor do quartil 75% à nova categoria criada no passo anterior.

In [ ]:
df_nivel6.loc[df_nivel6['Valor Pago Sinistro'] > 38423.58, 'Categoria Sinistro'] = 'Nível 6 Intermediário'


Agora, iremos criar uma nova variável para localizar justamente esses valores para que possemos plotar um boxplot e verificar se há a valores discrepantes.

In [ ]:
df_nivel6_intermed = df_nivel6.loc[(df_nivel6['Categoria Sinistro'] == 'Nível 6 Intermediário') ]
df_nivel6_intermed.boxplot(['Valor Pago Sinistro'])


A partir disso, dentro na nova categoria criada, iremos calcular de novo o valor do quartil para que possemos eliminar todos os valores acima disso, considerados outliers.

In [ ]:
quartil = df_nivel6_intermed['Valor Pago Sinistro'].quantile(0.75)

quartil

Então, iremos armazenar os valores superiores ao valor do quartil 75% em uma nova variável.

In [ ]:
df_nivel6_intermed_maiores = df_nivel6_intermed.loc[(df_nivel6_intermed['Valor Pago Sinistro'] > 68323.55) ]

df_nivel6_intermed_maiores ['Categoria Sinistro'].value_counts()

Agora, finalmente, podemos retirar esses valores recém localizados do banco de dados, pois eles são considerado outliers e podem atrapalhar nas análises que irão ser feitas. Para fazer esse processo, utilizaremos do método drop, que usará o parâmetro `.index` para localizar os outliers pelo seu índice, e então, eliminá-los.

In [ ]:
df = df.drop(df_nivel6_intermed_maiores.index)

df

A seguinte célula mostra os 10 maiores valores em 'Valor Pago Sinistro' após a retirada de outliers, através do método `nlargest()`.

In [ ]:
df.nlargest(10, 'Valor Pago Sinistro')

A partir de agora, iremos fazer o mesmo processo para as outras categorias.

In [ ]:
df_nivel5 = df.loc[(df['Categoria Sinistro'] == 'Nível 5') ]
quartil = df_nivel5['Valor Pago Sinistro'].quantile(0.75)

quartil


In [ ]:
df_nivel5['Categoria Sinistro'] = df_nivel5['Categoria Sinistro'].astype('category')

df_nivel5['Categoria Sinistro'] = df_nivel5['Categoria Sinistro'].cat.add_categories(['Nível 5 Intermediário'])

In [ ]:
df_nivel5.loc[df_nivel5['Valor Pago Sinistro'] > 1543.435, 'Categoria Sinistro'] = 'Nível 5 Intermediário'

df_nivel5_intermed = df_nivel5.loc[(df_nivel5['Categoria Sinistro'] == 'Nível 5 Intermediário') ]

df_nivel5_intermed.boxplot(['Valor Pago Sinistro'])

In [ ]:
df_nivel4 = df.loc[(df['Categoria Sinistro'] == 'Nível 4') ]
quartil = df_nivel4['Valor Pago Sinistro'].quantile(0.75)

quartil

In [ ]:
df_nivel4['Categoria Sinistro'] = df_nivel4['Categoria Sinistro'].astype('category')

df_nivel4['Categoria Sinistro'] = df_nivel4['Categoria Sinistro'].cat.add_categories(['Nível 4 Intermediário'])

In [ ]:
df_nivel4.loc[df_nivel4['Valor Pago Sinistro'] > 415.74, 'Categoria Sinistro'] = 'Nível 4 Intermediário'

df_nivel4_intermed = df_nivel4.loc[(df_nivel4['Categoria Sinistro'] == 'Nível 4 Intermediário') ]

df_nivel4_intermed.boxplot(['Valor Pago Sinistro'])

In [ ]:
df_nivel3 = df.loc[(df['Categoria Sinistro'] == 'Nível 3') ]
quartil = df_nivel3['Valor Pago Sinistro'].quantile(0.75)

quartil

In [ ]:
df_nivel3['Categoria Sinistro'] = df_nivel3['Categoria Sinistro'].astype('category')

df_nivel3['Categoria Sinistro'] = df_nivel3['Categoria Sinistro'].cat.add_categories(['Nível 3 Intermediário'])

In [ ]:
df_nivel3.loc[df_nivel3['Valor Pago Sinistro'] > 150.02, 'Categoria Sinistro'] = 'Nível 3 Intermediário'

df_nivel3_intermed = df_nivel3.loc[(df_nivel3['Categoria Sinistro'] == 'Nível 3 Intermediário') ]

df_nivel3_intermed.boxplot(['Valor Pago Sinistro'])

A partir do nível 2, iremos apenas plotar o boxplot, sem criar uma categoria intermediária, visto que são categorias que possem intervalos menores.

In [ ]:
df_nivel2 = df.loc[(df['Categoria Sinistro'] == 'Nível 2') ]

df_nivel2.boxplot(['Valor Pago Sinistro'])


In [ ]:
df_nivel1 = df.loc[(df['Categoria Sinistro'] == 'Nível 1') ]

df_nivel1.boxplot(['Valor Pago Sinistro'])


In [ ]:
df['Categoria Sinistro'].value_counts()

Comparando com a média anterior, o novo valor condiz muito mais com a frequência de valores da coluna.

In [ ]:
df['Valor Pago Sinistro'].mean()

### Normalização dos dados
Com intuito de normalizar o 'Valor Pago Sinistro' e deixar a tabela ainda mais condizente com a frequência real de valores, criaremos uma coluna 'Normalização Valor Pago Sinistro' que conterá os valores normalizados, utilizando do método Z-Score, que possue a seguinte fórmula:
\begin{align}
Z = \frac{x - \mu}{\sigma}
\end{align}

In [ ]:
df['Normalização Valor Pago Sinistro'] = (df['Valor Pago Sinistro'] - df['Valor Pago Sinistro'].mean()) / df['Valor Pago Sinistro'].std()

### Remoção de valores duplicados
Agora, como último passo da limpeza dos dados, iremos retirar as linhas que estão duplicadas, pois esses valores também podem ser considerados dados ruidosos. Então, utilizaremos o método .shape[0] para contar as linhas do banco atual. Faremos a limpeza de duplicados com o método .drop_duplicates() com o método "keep" sendo igual a "last", pois ele irá manter a última ocorrência da duplicada e retirará a outras.


In [ ]:
df.shape[0]

In [ ]:
df.drop_duplicates(keep='last')

Após essa retirada, conferiremos o número de linhas do banco.

In [ ]:
df.shape[0]

Todo esse processo demonstra a importância de uma limpeza de dados eficiente para garantir a qualidade das análises subsequentes. A remoção de dados duplicados, tratamento de valores ausentes ou muito discrepantes, normalização das variáveis e criação de categorias são etapas cruciais para reduzir ruídos e distorções no conjunto de dados. Ao realizar essas transformações, garantimos que o modelo seja mais confiável, resultando em previsões precisas para a tomada de decisões.